## This notebook demonstrates how to calculate `optical flow` with OpenCV.
***

### Optical flow is the pattern of apparent motion of image objeccts between two consecutive frames caused by the movement of an object or a camera.

### It is a 2D vector field where each vector is a displacement of vector showing the movement of points from the first frame to the second.

![optical flow](inputs/optical_flow_basic1.jpg)

### The image above shows a ball moving in 5 consecutive frames.

#### Optical flow has many applications in areas like:
- Structure from motion.
- Video compression
- Video stabilization.

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture('inputs/slow_traffic_small.mp4')

In [3]:
#Parameters for ShiTomasi corner detection.
feature_params = dict(maxCorners = 180, 
qualityLevel = 0.3, minDistance = 7, 
blockSize = 7)


#Parameters for Lucas Kanade optical flow.
lk_parameters = dict(winSize = (15, 15),
maxLevel = 2, 
criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [4]:
#Create some random colors.
color = np.random.randint(0, 255, (100, 3))

#Take the first frame and find corners in it.
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#Create a mask image for drawing purposes.
mask = np.zeros_like(old_frame)

In [ ]:
while(1):
    ret, frame = cap.read()
    if not ret:
        print('No frames grabbed!')
        break

    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #Calculate the optical flow.
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_parameters)

    #Select the good parameters.
    if p1 is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]

    #Draw the tracks.
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    img = cv2.add(frame, mask)

    cv2.imshow('frame', img)

    #Break gracefully.
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break

    #Update the previous frame and the previous points.
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()